In [6]:
from confluent_kafka import Producer
import socket

conf = {'bootstrap.servers': "localhost:9092",
        'client.id': socket.gethostname()}

producer = Producer(conf)

In [34]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "foo",
        'auto.offset.reset': 'smallest'}

consumer = Consumer(conf)

In [53]:
topic = "multimedia2"

In [15]:
producer.produce(topic, key="key", value="value")

In [16]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

producer.produce(topic, key="key", value="value", callback=acked)

# Wait up to 1 second for events. Callbacks will be invoked during
# this method call if the message is acknowledged.
producer.poll(1)

Message produced: <cimpl.Message object at 0x7f839811d840>


3

In [19]:
from confluent_kafka import KafkaError, KafkaException,Message
import sys


In [20]:
def msg_process(msg):
    print(f"key: {msg.key().decode('utf-8')}, value: {msg.value().decode('utf-8')}")

In [45]:
import json

for n in range(10):
    record_key = f"alice"
    record_value = json.dumps({'count': n})
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value, on_delivery=acked)
    # p.poll() serves delivery reports (on_delivery)
    # from previous produce() calls.
    producer.poll(0)


Producing record: alice	{"count": 0}
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Message produced: <cimpl.Message object at 0x7f83811d88c0>
Producing record: alice	{"count": 1}
Producing record: alice	{"count": 2}
Producing record: alice	{"count": 3}
Producing record: alice	{"count": 4}
Producing record: alice	{"count": 5}
Producing record: alice	{"count": 6}
Producing record: alice	{"count": 7}
Producing record: alice	{"count": 8}
Producing record: alice	{"count": 9}


In [52]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "foo",
        'auto.offset.reset': 'smallest'}

consumer = Consumer(conf)

In [54]:
consumer.subscribe([topic])
    # Process messages
total_count = 0
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            # No message available within timeout.
            # Initial message consumption may take up to
            # `session.timeout.ms` for the consumer group to
            # rebalance and start consuming
            print("Waiting for message or event/error in poll()")
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
        else:
            # Check for Kafka message
            record_key = msg.key()
            record_value = msg.value()
            data = json.loads(record_value)
            count = data['count']
            total_count += count
            print("Consumed record with key {} and value {}, \
                    and updated total count to {}"
                    .format(record_key, record_value, total_count))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()

Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Consumed record with key b'alice' and value b'{"count": 0}',                     and updated total count to 0
Consumed record with key b'alice' and value b'{"count": 1}',                     and updated total count to 1
Consumed record with key b'alice' and value b'{"count": 2}',                     and updated total count to 3
Consumed record with key b'alice' and value b'{"count": 3}',                     and updated total count to 6
Consumed record with key b'alice' and value b'{"count": 4}',                     and updated total count to 10
Consumed record with key b'alice' and value b'{"count": 5}',                     and updated total count to 15
Consumed record with key b'alice' and value b'{"count": 6}',                     and updated total count to 21
Consumed record with key b'alice' and value b'{

In [29]:
basic_consume_loop(consumer, [topic])

AttributeError: 'NoneType' object has no attribute 'decode'

In [23]:
from confluent_kafka import Consumer

def commit_completed(err, partitions):
    if err:
        print(str(err))
    else:
        print("Committed partition offsets: " + str(partitions))

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "foo",
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'on_commit': commit_completed}

consumer = Consumer(conf)